# Tone Spiral


Brett Deaton - Fall 2023


A representation of pure tones in shapes and sounds.


The class structure is:

- Scale
  - EqualTempered
    - Chromatic
    - Major
    - Minor
    - Pentatonic
    - GreekMode
    - ...
  - JustIntonation
    - Pythagorean
    - FiveLimit
    - Harmonic
    - ...
  - Free
  - Raga (?)
  - Gamelan (?)
  - ...


## Usage


In [ ]:
# scale = Chromatic("Ab")
scale = Major("C")
print(scale)
# play(scale)
# draw(scale)

## Definitions


In [ ]:
class Scale:
    """A series of tones in a distinct order.

    Typical usage of x = Scale() includes print(x), which displays the
    primary frequencies in Hz.
    """

    def __init__(self):
        self.primaries = None

    def __str__(self):
        return " ".join([f"{freq:.2f}" for freq in self.primaries])


class EqualTempered(Scale):
    """Tones from the traditional tuning of a piano.

    The frequencies live on a grid equally-spaced in log frequency,
    with 12 pitches (aka semitones) between each octave. The default grid
    is referenced to the standard pitch of A4=440 Hz, but may be set to
    any frequency at construction.
    """

    _FREQ_A4 = 440  # default reference frequency

    def __init__(self, key_name, ref_freq=_FREQ_A4):
        """Initialize

        Args:
            key_name (str): the 'home' note of the scale; allowed values
                are any traditional western keys,
                expressed as 1 or 2 characters; e.g. Ab, A, A#, etc.
            ref_freq (float): the standard pitch in Hz used to compute the
                grid of tones; it's the reference point, but may or may
                not be included in the scale; default=440
        """
        super().__init__()
        self._NAMES_FLATS = ("C", "Db", "D", "Eb", "E", "F",
                             "Gb", "G", "Ab", "A", "Bb", "B")
        self._NAMES_SHARPS = ("C", "C#", "D", "D#", "E", "F",
                              "F#", "G", "G#", "A", "A#", "B")
        self._FREQ_C4 = self.ith_freq_from_primary(ref_freq, -9)
        self._FREQS = tuple(self.ith_freq_from_primary(self._FREQ_C4, i)
                            for i in range(12))

    def freq_from_name(self, name, octave=4):
        """Lookup the frequency associated with a traditional western note name.

        Args:
            name (str): the name of the note,
                expressed as 1 or 2 characters; e.g. Ab, A, A#, etc.
            octave (int): the octave to draw from in scientific pitch notation;
                for example the 4th octave beginning at Middle C covers
                approximately 262-523 Hz; default=4
        """
        index = self._NAMES_FLATS.index(name)
        if index is None:
            index = self._NAMES_SHARPS.index(name)
            if index is None:
                raise ValueError(f"Name {name} is not a scale name")
        return self._FREQS[index] * 2**(octave-4)

    def ith_freq_from_primary(self, primary, i):
        """Compute the frequency of the tone i semitones above the given primary.

        The tuning is equal-tempered yielding 12 semitones to an octave.

        Args:
            primary (float): reference frequency in Hz
            i (int): number of half-steps above primary,
                indexed from 0, negative i means step down

        Returns:
            the computed frequency (float)
        """
        return primary * pow(2, i/12)


class Chromatic(EqualTempered):
    """The 12 notes from a piano keyboard."""

    def __init__(self, key_name, ref_freq=EqualTempered._FREQ_A4):
        """Initialize. See args for EqualTempered."""
        super().__init__(key_name, ref_freq)
        self.key_name = key_name
        self.principle = self.freq_from_name(key_name)
        self.primaries = tuple(self.ith_freq_from_primary(self.principle, i)
                               for i in range(12))


class Major(EqualTempered):
    """The seven notes from a major scale."""

    def __init__(self, key_name, ref_freq=EqualTempered._FREQ_A4):
        """Initialize. See args for EqualTempered."""
        super().__init__(key_name, ref_freq)
        self.key_name = key_name
        self.principle = self.freq_from_name(key_name)
        self.primaries = tuple(self.ith_freq_from_primary(self.principle, i)
                               for i in (0, 2, 4, 5, 7, 9, 11))

In [ ]:
def play(scale):
    pass

## Scratch


In [ ]:
import plotly.express as px
import pandas as pd
data = {'freq': [1, 12.1], 'rad': [450, 333]}
df = pd.DataFrame(data=data)
fig = px.scatter_polar(df, r="rad", theta="freq")
fig.show()

In [ ]:
# fails because musicpy depends on simpleaudio which I can't get installed
import musicpy as mp
guitar = (mp.C('CM7', 3, 1/4, 1/8) ^ 2 |
          mp.C('G7sus', 2, 1/4, 1/8) ^ 2 |
          mp.C('A7sus', 2, 1/4, 1/8) ^ 2 |
          mp.C('Em7', 2, 1/4, 1/8) ^ 2 |
          mp.C('FM7', 2, 1/4, 1/8) ^ 2 |
          mp.C('CM7', 3, 1/4, 1/8)@1 |
          mp.C('AbM7', 2, 1/4, 1/8) ^ 2 |
          mp.C('G7sus', 2, 1/4, 1/8) ^ 2) * 2

mp.play(guitar, bpm=100, instrument=25)

In [ ]:
# fails to make any sounds
import time
import rtmidi

midiout = rtmidi.MidiOut()
available_ports = midiout.get_ports()

if available_ports:
    midiout.open_port(0)
else:
    midiout.open_virtual_port("My virtual output")

with midiout:
    note_on = [0x90, 60, 112]  # channel 1, middle C, velocity 112
    note_off = [0x80, 60, 0]
    midiout.send_message(note_on)
    time.sleep(0.5)
    midiout.send_message(note_off)
    time.sleep(0.1)

del midiout

## Notes Etc


#### Nice Resources

- [muted.io](https://muted.io/): lots of interactive music theory tools


#### Useful libraries

##### Play

- [pyaudio](https://people.csail.mit.edu/hubert/pyaudio/): play or record
  various formats (trouble installing, see below)
- [simpleaudio](https://pypi.org/project/simpleaudio/): don't know, looks okay
  (trouble installing, see below)
- [musicpy](https://pypi.org/project/musicpy/): program notes and songs using
  the abstraction of music theory (trouble running because it depends on
  simpleaudio, see below)
- [python-rtmidi](https://pypi.org/project/python-rtmidi/): low-level MIDI
  programming, with youtube tutorial
  [Programming with MIDI in Python](https://www.youtube.com/watch?v=JYslZkc90GI)
- [pymidi](https://pypi.org/project/pymidi/): don't know, looks okay

##### Draw

- plotly
- matplotlib
- [drawsvg](https://pypi.org/project/drawsvg/): well-documented, and
  community-supported
- [svgwrite](https://pypi.org/project/svgwrite/): got some longevity, and
  well-documented, but no longer expanded, just bugfixes
- [svg.py](https://pypi.org/project/svg.py/): basic svg support


#### TODO


#### Next tasks

Somewhat in order of operation, first at top

- Figure out how to make ith_freq_from_primary() a static class method of
  Scale, and also make other stuff static that should be
- Expand Scale to provide an interface that returns the frequencies (?)
- Create a draw function that converts a Scale's frequencies to a DataFrame
  appropriate to plot in a plotly scatter_polar plot


##### C-based libraries fail to install

Pip fails to install some audio libraries with `subprocess-exited-with-error`.
These libraries include c code, and so require a compilation step.

Steps to reproduce:

1. In virtual env or locally
2. `pip install pyaudio` or `simpleaudio`

Expected output: "Successfully installed ..."

Actual output (for pyaudio):

```
src/pyaudio/device_api.c:9:10: fatal error: portaudio.h: No such file or directory
compilation terminated.
error: command '/usr/bin/x86_64-linux-gnu-gcc' failed with exit code 1
```

Actual output (for simpleaudio):

```
c_src/simpleaudio_alsa.c:8:10: fatal error: alsa/asoundlib.h: No such file or directory
compilation terminated.
error: command '/usr/bin/x86_64-linux-gnu-gcc' failed with exit code 1
```

Possible solutions:

- These are old packages (?), with some other warnings on install too. Just
  find modern better packages.
- Install python developer tools, according to the article
  [How to fix ...](https://www.geeksforgeeks.org/how-to-fix-fatal-error-python-h-no-such-file-or-directory/).
  (Didn't fix it on a quick trial, I've already got python3-dev installed.)
  Checkout this [stackoverflow](https://stackoverflow.com/questions/21530577/fatal-error-python-h-no-such-file-or-directory).


##### Runtime error with alsa-lib

Steps to reproduce:

1. Run boilerplate code from rtmidi pypi page

Expected output: pretty sounds

Actual output:

```
SystemError: MidiOutAlsa::initialize: error creating ALSA sequencer client object.
```

Possible solutions:

- Link the library so it's visible to the library. See
  [github issue](https://github.com/SpotlightKid/python-rtmidi/issues/138)
  ```
  $ sudo mkdir usr/lib/alsa-lib
  $ sudo ln -s /usr/lib/x86_64-linux-gnu/alsa-lib/* /usr/lib/alsa-lib/
  $ # or rather, link into /usr/lib64/alsa-lib/ if that's what the error message is
  ```
  this resolved the SystemError, but still didn't play any sounds
